<a href="https://colab.research.google.com/github/vlasvlasvlas/Colabs/blob/main/Dynamic_Barchart_Index_Audit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Comprehensive Index Calculation Script

-  This script provides a complete setup for loading, processing, and visualizing data for index calculation.

-  It utilizes various libraries including pandas, matplotlib, and seaborn to facilitate data manipulation and graphical representation. Widgets are used for interactive input and control.


In [ ]:
# Comprehensive Index Calculation Script
# This script provides a complete setup for loading, processing, and visualizing data for index calculation.
# It utilizes various libraries including pandas, matplotlib, and seaborn to facilitate data manipulation
# and graphical representation. Widgets are used for interactive input and control.

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
import numpy as np
import io
from ipywidgets import widgets, VBox, HBox, Button, HTML, Layout
from google.colab import files
from IPython.display import display, clear_output # clear plot


# Define a layout for the input widgets to increase their width
input_layout = Layout(width='500px')  # Adjust width as needed

# Create widgets for input values and error message with adjusted layout
inputvalue1 = widgets.FloatText(value=30, description='InputValue1:', style={'description_width': 'initial'}, layout=input_layout)
inputvalue2 = widgets.FloatText(value=30, description='InputValue2:', style={'description_width': 'initial'}, layout=input_layout)
inputvalue3 = widgets.FloatText(value=10, description='InputValue3:', style={'description_width': 'initial'}, layout=input_layout)
inputvalue4 = widgets.FloatText(value=30, description='InputValue4:', style={'description_width': 'initial'}, layout=input_layout)
error_message = HTML(value="")

# Button to load file and update the chart
upload_button = Button(description="Load CSV File", layout=Layout(width='auto', margin='10px 0 10px 0'))
update_button = Button(description="Update Chart", layout=Layout(width='auto', margin='10px 0 0 0'))

# Container for the filename
filename_label = HTML(value="No file uploaded.")

def upload_file(_):
    global data, filename  # Declare as global if you need to use these variables outside of this function
    uploaded = files.upload()
    if uploaded:
        filename = next(iter(uploaded))
        data = pd.read_csv(io.BytesIO(uploaded[filename]))
        filename_label.value = f"File uploaded: {filename}"

        # Check if data has enough columns
        if data.shape[1] < 5:  # Assuming 'country' plus at least 4 data columns
            error_message.value = "Error: The file does not contain enough columns."
            return
        else:
            error_message.value = ""

        # Update widget descriptions with column names
        inputvalue1.description = f'InputValue1 ({data.columns[1]})'
        inputvalue2.description = f'InputValue2 ({data.columns[2]})'
        inputvalue3.description = f'InputValue3 ({data.columns[3]})'
        inputvalue4.description = f'InputValue4 ({data.columns[4]})'

        update_graph(None)  # Call to update the graph directly after loading

# Function to update the chart and validate the sum
def update_graph(_):

    # Clear the current figure
    plt.clf()  # Clear the current figure
    plt.close()  # Close the plotting window
    clear_output(wait=True)  # Clear the output of the current cell and wait for new output

    # need nada to plot
    if 'data' not in globals():
        error_message.value = "Please upload a file first."
        return

    # check total input <= 100:
    total_input_value = inputvalue1.value + inputvalue2.value + inputvalue3.value + inputvalue4.value

    if total_input_value > 100:
        error_message.value = "Error: The sum of the values must not exceed 100. Currently sum: {:.2f}".format(total_input_value)
        return
    else:
        error_message.value = ""

    # creates index column
    data['index'] = (data.iloc[:,1] * inputvalue1.value/100 +
                      data.iloc[:,2] * inputvalue2.value/100 +
                      data.iloc[:,3] * inputvalue3.value/100 +
                      data.iloc[:,4] * inputvalue4.value/100 )

    # get first 50 rows (countries) order by index column
    top_50 = data.sort_values('index', ascending=False).head(50)

    # adds the number of order
    top_50['country'] = top_50['country'].str.slice(0, 40) + ' (' + (top_50.reset_index().index + 1).astype(str) + ')'

    # starts plotting
    plt.figure(figsize=(12, 15))
    color_palette = 'coolwarm'  # https://seaborn.pydata.org/tutorial/color_palettes.html
    bar_plot = sns.barplot(x='index', y='country', data=top_50, palette=color_palette)

    # for each value in top 50 countries index based:
    for index, value in enumerate(top_50['index']):
        text = bar_plot.text(value, index, f'{value:.2f}', color='black', va='center', fontweight='bold', ha='right')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])

    # chart labels
    plt.title(f'Index Calculation for Data Analysis ({filename})')
    plt.xlabel('Index')
    plt.ylabel('Country')
    plt.xlim(0, top_50['index'].max() * 1.1)

    # render plot
    upload_button = Button(description="Load new CSV File", layout=Layout(width='auto', margin='10px 0 10px 0'))
    input_widgets = VBox([title, description, filename_label, upload_button, inputvalue1, inputvalue2, inputvalue3, inputvalue4, update_button, error_message])
    display(input_widgets)
    plt.show()


# Link the button with the file loading and update functions
upload_button.on_click(upload_file)
update_button.on_click(update_graph)

# Widgets for title and description
title = HTML(value='<h1>Index Calculation for Data Analysis</h1>', layout=Layout(margin='0 0 5px 0'))

description = HTML(value='''
Analysis Process
Follow these steps to calculate the index and view the results:

     Upload the CSV file using the 'Upload File' button.
     Set the input values for the four indicators: InputValue1, InputValue2, InputValue3, and InputValue4.
     Press the 'Refresh Chart' button to view the bar graph of the uploaded data.

Make sure that each file contains at least the five necessary columns: 'country', and the 4 columns of data for which the index needs to be generated.
<hr>
''', layout=Layout(margin='0 0 15px 0'))


# Display the widgets and messages
input_widgets = VBox([title, description, filename_label, upload_button,  error_message])
display(input_widgets)
